In [2]:
# imports and dependencies, api_key

In [3]:
# file path
sf_crime = "http://opendata.mybluemix.net/crimes?lat=37.7798&lon=-122.3939&radius=200"
chi_crime = "http://opendata.mybluemix.net/crimes?lat=41.8986&lon=-87.6628&radius=200"
vegas_crime = "http://opendata.mybluemix.net/crimes?lat=36.067&lon=-115.173&radius=800"

In [4]:
# initial data read

In [5]:
# formatting of data, NaN handling

In [6]:
# preview of data

## Subject Headline

### Subject focus

trend1 description / question / prompt

In [7]:
# pull above data - CHANGE data REFERENCE ( don't rewrite master copy of data )

In [12]:
# re-format/style data for trend1

In [9]:
# manipulate data
# setup analysis

In [10]:
# preview of data

In [11]:
# visualization of data
# consider the best visualization for what you want to show

In [13]:
# free space

trend2 description / question / prompt

In [14]:
# re-format/style data for trend2

In [15]:
# manipulate data
# setup analysis

In [16]:
# preview of data

In [18]:
# visualization of data
# consider the best visualization for what you want to show

In [19]:
# free space

trend3 description / question / prompt

In [22]:
# re-format/style data for trend3

In [23]:
# manipulate data
# setup analysis

In [24]:
# preview of data

In [25]:
# visualization of data
# consider the best visualization for what you want to show

Any comments on observations?

## Subject Headline

### Subject focus

trend1 description / question / prompt

In [7]:
# pull above data - CHANGE data REFERENCE ( don't rewrite master copy of data )

In [12]:
# re-format/style data for trend1

In [9]:
# manipulate data
# setup analysis

In [10]:
# preview of data

In [11]:
# visualization of data
# consider the best visualization for what you want to show

In [13]:
# free space

trend2 description / question / prompt

In [14]:
# re-format/style data for trend2

In [15]:
# manipulate data
# setup analysis

In [16]:
# preview of data

In [18]:
# visualization of data
# consider the best visualization for what you want to show

In [19]:
# free space

trend3 description / question / prompt

In [22]:
# re-format/style data for trend3

In [23]:
# manipulate data
# setup analysis

In [24]:
# preview of data

In [27]:
# visualization of data
# consider the best visualization for what you want to show

Any comments on observations?

## Subject Headline

### Subject focus

trend1 description / question / prompt

In [7]:
# pull above data - CHANGE data REFERENCE ( don't rewrite master copy of data )

In [12]:
# re-format/style data for trend1

In [9]:
# manipulate data
# setup analysis

In [10]:
# preview of data

In [11]:
# visualization of data
# consider the best visualization for what you want to show

In [13]:
# free space

trend2 description / question / prompt

In [14]:
# re-format/style data for trend2

In [15]:
# manipulate data
# setup analysis

In [16]:
# preview of data

In [18]:
# visualization of data
# consider the best visualization for what you want to show

In [19]:
# free space

trend3 description / question / prompt

In [22]:
# re-format/style data for trend3

In [23]:
# manipulate data
# setup analysis

In [24]:
# preview of data

In [26]:
# visualization of data
# consider the best visualization for what you want to show

Any comments on observations?